In [1]:
# Init
import os
import glob
import sys
import pickle

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import numpy as np
import pandas as pd
import warnings
import multiprocessing
np.set_printoptions(suppress=True)
from numpy import concatenate as c
from numpy import cumsum
import random

# Plotting imports
import altair as alt

In [2]:
# Path to results
resultDir = "/home/siddharth/Documents/Projects/Simulicron/Project_1C/etaVsTR/"

# Add them to a list
resultList = []
for filename in glob.glob(os.path.join(resultDir, '*.pickle')):
    with open(filename, 'rb') as f:
        resultList.append(pickle.load(f))

# Store results in a dataframe
resultDataframe = pd.DataFrame(resultList)

In [8]:
# Create a copy for inplace handling
resultDataframeC = resultDataframe.copy()

In [9]:
# Perform some data wrangling
resultDataframeC[['TranspositionRate_1','TranspositionRate_2']] = pd.DataFrame(resultDataframeC.TRate.tolist(), index= resultDataframeC.index)
resultDataframeC[['TE_1_CN','TE_2_CN']] = resultDataframeC['TEfamilyCN'].apply(pd.Series)
resultDataframeC[['TE_1_RG','TE_2_RG']] = resultDataframeC['TEfamilyRg'].apply(pd.Series)

In [11]:
resultDataframeC['State'].value_counts()

FLUX    954
Name: State, dtype: int64

In [12]:
resultDataframeC

,State,Generatrion,NTE,AvgCopyNum,CopyNumVar,TEfamilyCN,TEfamilyVR,TEfamilyRg,AvgFit,HGTGen,...,Freq,TRate,Tau,selPen,TranspositionRate_1,TranspositionRate_2,TE_1_CN,TE_2_CN,TE_1_RG,TE_2_RG
0,FLUX,10001,645836,"[1.0, 1.094, 1.154, 1.272, 1.387, 1.504, 1.556...","[0.994, 1.225164, 1.4422840000000001, 1.652015...","{1: (1.0, 1.094, 1.154, 1.272, 1.387, 1.504, 1...","{1: ([0.994], [1.225164], [1.4422840000000001]...","{1: (0.0, 0.003, 0.005, 0.016, 0.014, 0.023, 0...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,"[1.0, 0.8078139560616969]","[0.07321051887859287, 0.12623607707425333]",1.0,0.0,0.073211,0.126236,"(1.0, 1.094, 1.154, 1.272, 1.387, 1.504, 1.556...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.003, 0.005, 0.016, 0.014, 0.023, 0.019...","(0.0, 0.0006568000968027811, 0.001094666828004..."
1,FLUX,10001,569583,"[1.0, 1.991, 3.917, 7.354, 13.169, 22.188, 33....","[1.026, 3.820919000000001, 10.276110999999998,...","{1: (1.0, 1.991, 3.917, 7.354, 13.169, 22.188,...","{1: ([1.026], [3.820919000000001], [10.2761109...","{1: (0.0, 0.037, 0.096, 0.18, 0.285, 0.473, 0....","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,"[1.0, 0.8010976206287925]","[0.9679028824159526, 0.6782385628448176]",1.0,0.0,0.967903,0.678239,"(1.0, 1.991, 3.917, 7.354, 13.169, 22.188, 33....","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.037, 0.096, 0.18, 0.285, 0.473, 0.642,...","(0.0, 0.020158150561700863, 0.0523022284844130..."
2,FLUX,10001,431139,"[1.0, 1.972, 3.909, 7.536, 14.055, 23.743, 35....","[0.978, 3.7432160000000003, 11.454719, 28.1187...","{1: (1.0, 1.972, 3.909, 7.536, 14.055, 23.743,...","{1: ([0.978], [3.7432160000000003], [11.454719...","{1: (0.0, 0.014, 0.043, 0.132, 0.302, 0.48, 0....","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,"[1.0, 0.48754783875693075]","[0.9960450089563727, 0.10191890402231707]",1.0,0.0,0.996045,0.101919,"(1.0, 1.972, 3.909, 7.536, 14.055, 23.743, 35....","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.014, 0.043, 0.132, 0.302, 0.48, 0.686,...","(0.0, 0.007151768694776185, 0.0219661467053840..."
3,FLUX,10001,437367,"[1.0, 1.444, 1.966, 2.512, 3.34, 4.633, 6.159,...","[0.94, 2.016864, 3.272844, 4.903855999999999, ...","{1: (1.0, 1.444, 1.966, 2.512, 3.34, 4.633, 6....","{1: ([0.94], [2.016864], [3.272844], [4.903855...","{1: (0.0, 0.009, 0.022, 0.047, 0.065, 0.086, 0...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,"[1.0, 0.28894684505770607]","[0.3802660756748293, 0.4212077166174484]",1.0,0.0,0.380266,0.421208,"(1.0, 1.444, 1.966, 2.512, 3.34, 4.633, 6.159,...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.009, 0.022, 0.047, 0.065, 0.086, 0.136...","(0.0, 0.0054883807363089385, 0.013416041799866..."
4,FLUX,10001,698684,"[1.0, 1.922, 3.578, 6.159, 10.872, 18.213, 27....","[1.012, 4.115916, 11.353916000000002, 22.68371...","{1: (1.0, 1.922, 3.578, 6.159, 10.872, 18.213,...","{1: ([1.012], [4.115916], [11.353916000000002]...","{1: (0.0, 0.028, 0.086, 0.159, 0.29, 0.451, 0....","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,"[1.0, 0.9430843128401835]","[0.9545990537606484, 0.3236547627481926]",1.0,0.0,0.954599,0.323655,"(1.0, 1.922, 3.578, 6.159, 10.872, 18.213, 27....","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.028, 0.086, 0.159, 0.29, 0.451, 0.611,...","(0.0, 0.004131490675001559, 0.0126895785017905..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,FLUX,10001,322580,"[1.0, 1.047, 1.143, 1.26, 1.401, 1.627, 1.844,...","[1.012, 1.342791, 1.5985509999999998, 1.9144, ...","{1: (1.0, 1.047, 1.143, 1.26, 1.401, 1.627, 1....","{1: ([1.012], [1.342791], [1.5985509999999998]...","{1: (0.0, 0.005, 0.007, 0.008, 0.014, 0.021, 0...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,"[1.0, 0.7478419926901018]","[0.1132166720092449, 0.7399200439469408]",1.0,0.0,0.113217,0.739920,"(1.0, 1.047, 1.143, 1.26, 1.401, 1.627, 1.844,...","(0.0, 